In [1]:
from __future__ import division
import PyDSTool as dst
import numpy as np

#PyDSTool ModelSpec Tutorial

This tutorial is based on the following PyDSTool documentation pages:
* http://www.ni.gsu.edu/~rclewley/PyDSTool/ModelSpec.html
* http://www.ni.gsu.edu/~rclewley/PyDSTool/Symbolic.html


The goal of the ModelSpec classes is to provide a class hierarchy that allows complex dynamical models to be composed from simple units of mathematical expression.

The `Quantity` classes are the elementary model building blocks of PyDSTool --- the ultimate "leaves" in the hierarchy of model specification that is supported by the ModelSpec classes. The essence of symbolic manipulation is the definition of Quantity objects. There are three types of Quantity objects: `Var`, `Par`, and `Fun`.

##`Var` (Variable)

Here is a variable declaration only using a name:

In [2]:
k = dst.Var('k')

Here is a variable declaration that includes a definition:

In [3]:
v = dst.Var('-k*(1 + k)*(1/2)', 'v')

The string representation of a variable is simply its name:

In [4]:
print v

v


In [5]:
print k

k


"Calling" a variable returns its definition:

In [6]:
print v()

-k*(1+k)*(1/2)


If a `Var` (or other `Quantity`, such as `Par` or `Fun`) does not have a definition, calling it returns its name once again:

In [7]:
print k()

k


At this time there is no class method to add the specification directly to a Quantity that has only been declared. (Such a method is not really necessary when the overhead in checking the specification is almost identical to that necessary to re-create the object entirely.)

##`QuantSpec` vs. `Quantity`

`v` is a `Var` object, which is a `Quantity` object:

In [77]:
v

Var v (ExpFuncSpec)

`v()` is a `QuantSpec` object:

In [80]:
v()

QuantSpec __result__ (ExpFuncSpec)

In the documentation, one might see references to `QuantSpec` or `Quantity` separately. In this tutorial, I will mostly use "definition" and "(`Quantity` object's) name" respectively.

##`Par` (parameter)

In [11]:
p = dst.Par('p')
print p

p


In [12]:
print p()

p


In [14]:
q = dst.Par('3.5e-3', 'p')
print q

p


In [15]:
print q()

3.5e-3


##`Fun` (function)

So far, the symbols we have seen appearing in the definition are 'free', and generally refer to other Quantities. They are not treated as 'local parameters', and therefore do not behave like arguments or inputs to a function.

In [16]:
v.freeSymbols

['k']

In [17]:
f = dst.Fun('(((k-(-10)))/(3+4))', [k], 'v')

In [18]:
f.freeSymbols

[]

Note that $k$ is a free symbol for `v`, but not for `f`. $k$ is an argument of `f`, and the code `[k]` defines `f`'s "signature". We can supply a value for $k$ during evaluation. Look carefully at the following examples.

Supplying a number as an argument:

In [21]:
print f(4)

2.0


Supplying a previously defined variable:

In [22]:
print f(k)

(k+10)/7


Supplying a previously defined parameter:

In [23]:
print f(p)

(p+10)/7


Supplying another previously defined parameter -- but note that so far we have been specifically supplying names, not definitions:

In [24]:
print f(q)

(p+10)/7


We can supply the definition by calling the `Quantity`:

In [25]:
print f(q())

1.4290714285714288


In [28]:
print f(p())

(p+10)/7


Once more comparing definitions vs. names:

In [29]:
print f(v())

(-k*(1+k)*0.5+10)/7


In [30]:
print f(v)

(v+10)/7


Too many arguments:

In [31]:
f(1, 2)

ValueError: Invalid number of arguments for auxiliary function

#Expression simplification and evaluation

##Simplification

Algebraic (mostly of brackets and signs) and arithmetical simplification of the definition of a `Quantity` is done using the object's `simplify` method. `simplify` works in-place on the object, and does not return anything.

In [39]:
u = dst.Var('((-k*(-k + 2*k)*(-(-1)/(1 + 1))))', 'u')

In [41]:
print u()

((-k*(-k+2*k)*(-(-1)/(1+1))))


In [42]:
u.simplify()

In [43]:
print u()

-k*(-k+2*k)*0.5


On the other hand, the object's `eval` method returns a simplified expression for the definition, without changing the original definition.

In [46]:
w = dst.Var('((-k*(-k + 2*k)*(-(-1)/(1 + 1))))', 'w')
print w.eval()

-k*(-k+2*k)*0.5


In [47]:
print w()

((-k*(-k+2*k)*(-(-1)/(1+1))))


##Evaluation

Quantities can also be evaluated at specific values for the free symbols, but care must be taken with functions in terms of the order in which the operations will be performed.

In [50]:
print w.eval(k = 1)

-0.5


Evaluations at non-numeric values for a symbol performs symbolic substitution:

In [52]:
print w.eval(k=v)

-v*(-v+2*v)*0.5


In [53]:
print w.eval(k=v())

-(-(-k*(1+k)*0.5)*(1-k*(1+k)*0.5)*0.5)*(-(-(-k*(1+k)*0.5)*(1-k*(1+k)*0.5)*0.5)+2*-(-k*(1+k)*0.5)*(1-k*(1+k)*0.5)*0.5)*0.5


In [54]:
print w.eval(k='j')

-j*(-j+2*j)*0.5


In [55]:
print w.eval(k=q)

-p*(-p+2*p)*0.5


In [56]:
print k()

k


In [57]:
print k

k


There is another function for expression substitution, called `subs`, that can act more "intelligently" (what is meant by this?) when dealing with defined `Par` quantities. It can also take a dictionary of assignments, and the result is always as simplified as possible.

In [63]:
print dst.subs(v(), {'k': 0})

0.0


Always remember to use `subs` on the right object. For instance, if we had used it on `v`'s name:

In [64]:
print dst.subs(v, {'k': 0})

v


This call did nothing because the `Quantity` assigned to Python name `'v'` evaluates to its name, which is also `'v'`. This name does not match any of the assignments in the call to `subs`, so no substitutions are made.

Evaluation of one symbolic expression can perform automatic substitution of values defined by expressions in the local scope:

In [69]:
q = dst.Quantity('xv+1','qv')
x = dst.Quantity('3','xv')
print q.eval()

4


In [70]:
a=x/q
print a

xv/qv


In [71]:
print a.eval()

0.75


Recall how we noted the difference between `Quantity` and `QuantSpec` objects previously:

In [82]:
a_ = x()/q()
print a_

3/(xv+1)


In other words, `a_` yields the division in terms of the definitions (or, `QuantSpec`), since we specifically called for them. We can use this difference between `Quantity` and `QuantSpec` to perform partial evaluations:

In [87]:
print a.eval() # using local scope information, xv = 3

0.75


In [85]:
print a.eval(qv=q())

xv/(xv+1)


In [83]:
print a.eval(xv=5,qv=q())  # override xv=3 from local scope

0.8333333333333334


Evaluation with an explicitly-supplied scope dictionary is much faster, if the relevant definitions are known. Equivalently, these definitions can be supplied in the call as a comma-separated sequence:

In [73]:
print a.eval(xv=x())

3/qv


In [74]:
print a.eval(xv=x(),qv=q())  # much faster

0.75


In [75]:
print a.eval({'xv': x, 'qv': q()})  # equivalently fast

0.75


##Advanced information about functions

In [88]:
# some variables
a = dst.Var('a')
b = dst.Var('b')
c = dst.Par('c')

# a function
gfun = dst.Fun(a+b/c, [a, b], 'g')

In [91]:
print gfun.eval(a=4)

ValueError: All function signature arguments are necessary for evaluation

In [92]:
print gfun.eval(a=4, b=3)

4+3/c


One might wonder why `c` was not required in the `eval` call. However, if we look at `gfun`'s signature, is a function of two arguments, `'a'` and `'b'`, while `'c'` is a free variable that *must refer to a parameter name* if the `Fun` object is to be used in the specification of a right-hand side (RHS), where RHS refers to the right hand side of a system of ODEs:

\begin{align}
    \vec{x} &= \left[\begin{array}{cccc}\dot{x}_1 & \dot{x}_2 & \dots & \dot{x}_n\end{array}\right] \\
    \quad \\
    \dot{x}_1 &= f_1(\vec{x}) \\
    \dot{x}_2 &= f_2(\vec{x}) \\
    \vdots \\
    \dot{x}_n &= f_n(\vec{x})
\end{align}

In [93]:
print gfun.eval(a=4,b=3,c=3)

5.0


In [94]:
print gfun(4,b)

4+b/c


`gfun(a, b)` evaluates to the `QuantSpec` object containing the definition of the function. As this is not a function object (`Fun`) any of the free names 'a', 'b', or 'c' can be evaluated using `eval`.

In [97]:
print gfun(a, b).eval(a=4)  # the result of gfun(a, b) is not a Fun, but a QuantSpec

4+b/c


Any evaluations given for names not present in the definition of the object will be ignored.

In [104]:
print gfun.eval(a=a, b=b, d=5, c=c)

a+b/c


##More symbolic substitutions: name maps

A more efficient way to make multiple symbolic substitutions which are solely textual, i.e. do not involve algebraic simplification, is to use the `mapNames(<name_mapping_dict>)` method of a `Quantity`. The method works in-place and does not return anything, and the `Quantity` whose method was called will change.

In [108]:
v.mapNames({'k': 'a/b'})
print v()

-a/b*(1+a/b)*0.5


This method takes a dictionary of `source: target` names. Matching only occurs on whole symbols (i.e. 'tokens'): a dictionary containing the mapping (`'a': 'b'`) will not map the symbol `'ka_2'` to `'kb_2'`.

The method will also map the `Quantity`'s name only if it matches a source name:

In [110]:
v.mapNames({'a': '44', '10': '-10'})
print v()

-44/b*(1+44/b)*0.5


# Symbolic versions of built-in math names

Symbolic versions of basic math functions (`pow`, `sqrt`, `abs`, etc.) and constants are exported by `Symbolic.py`. Importantly, they are title-cased, as in Maple. 

In [111]:
print v()

-44/b*(1+44/b)*0.5


In [126]:
print v.eval(b = 'Pow(1, 1)')

-990.0


In [127]:
print dst.subs(v(), {'b': 'Pow(x, y)'})

-44/Pow(x,y)*(1+44/Pow(x,y))*0.5


In [128]:
print dst.subs(v(), {'b': 'Pow(x, y)'}).eval(x = 1, y = 1)

-990.0


The functions supported included the trigonometric (`Sin`, `Cos`, `Tan`, but not inverse trigonometric functions), exponential (`Exp`) and logarithmic functions (`Log` -- why does this not work?), the absolute value (`Abs`), power (`Pow`), and square root (Sqrt) functions. The constants include `E` and `Pi`.



In [141]:
print v.eval(b = 'Sin(Tan(Sqrt(Abs(Pow((Cos(Pi*E)), 1)))))')

-1357.3899529842106


In [142]:
print v.eval(b = 'Sin(Tan(Sqrt(Abs(Pow(Log(Cos(Pi*E)), 1)))))')

-44/Sin(Tan(Sqrt(Abs(Pow(Log(Cos(3.141592653589793*2.718281828459045)),1)))))*(1+44/Sin(Tan(Sqrt(Abs(Pow(Log(Cos(3.141592653589793*2.718281828459045)),1))))))*0.5


In [146]:
import math
math.log(math.e)

1.0

#Multi-reference quantities

There is a way to specify a range of related Quantities at once, in a kind of macro. This is useful for repeated definitions, that perhaps are interconnected in a formulaic way. (You may be familiar with this kind of notation in the package XPP.)

In [148]:
v = dst.Var('v')
ipar = dst.Par('ipar')

z = dst.Var('3+v/((1+i)*ipar)', 'z[i,0,5]') # note z[i,0,5]

These define six `uantities`, `z0` to `z5`, that are referenced from the Python object `z` as `z[0]` to `z[5]`. 

In [152]:
print z[0]().eval()

3+1.71428571429/ipar


In [153]:
print z[1]().eval()

3+1.71428571429/(2*ipar)


The following example shows how to define a set of ODE right-hand sides that involve a circular connectivity pattern. Notice that the boundary variables `w0` and `w3` have to be defined separately, so that the multi-quantity definition refers to valid `Quantity` objects for all index values.

In [170]:
w0 = dst.Var('w0-2*(w3-w1)', 'w0', specType='RHSfuncSpec') # boundary value
w3 = dst.Var('w3-2*(w2-w0)', 'w3', specType='RHSfuncSpec') # boundary value
wi = dst.Var('w[i]-2*(w[i-1]-w[i+1])', 'w[i,1,2]', specType='RHSfuncSpec')

Alternatively, we could have used the modulo function (`%`) to give a cleaner definition:

In [169]:
for i in (np.arange(6) - 1):
    print "{}%3: {}".format(i, i%3)

-1%3: 2
0%3: 0
1%3: 1
2%3: 2
3%3: 0
4%3: 1


In [171]:
wi = dst.Var('w[i]-2*(w[(i-1)%3]-w[(i+1)%3])', 'w[i,0,3]', specType='RHSfuncSpec')

Note that the Python dereferencing of a multi-def `z` by (for example) `z[2]` actually creates an *instance* of `z2`. (What does this mean?)

In [174]:
print z[2]

z2


In [175]:
print z[2]()

3+v/((1+2)*ipar)


In [176]:
print z[0]

z2


In [177]:
print z[0]()

3+v/((1+0)*ipar)


In [178]:
print z[10]

IndexError: Index to multiple Quantity definition out of the valid range [0,5]

#More about the `QuantSpec` class

A `QuantSpec` object be considered a pre-`Quantity` form of raw symbolic expression, in that it has not yet been committed to a life as any particular Quantity sub-type (`Var`, `Par`, or `Fun`). It can be used in different ways in different `Quantity` definitions, and can be manipulated symbolically in its own right.

In [179]:
q = dst.QuantSpec('q', '1+k/2')

In [180]:
q

QuantSpec q (ExpFuncSpec)

In [181]:
q()

'1+k/2'

A major difference between a `Quantity` object and a `QuantSpec` object is that the `QuantSpec`'s name is not an important external part of its nature: only it's definition is important. Therefore, calling print on the object always returns its definition.

In [196]:
print q

1+k/2


In [197]:
print q()

1+k/2


In [195]:
print q.eval()

1+k/2


We can use a `QuantSpec` object to provide a definition for a `Quantity` object:

In [188]:
v = dst.Var(q, 'v')
v # this will print out the type of v

Var v (ExpFuncSpec)

In [189]:
v() # this will print out the type of v()

QuantSpec v (ExpFuncSpec)

In [190]:
(v())() # this will call the QuantSpec object returned by v()

'1+k/2'

In [191]:
w = dst.Var((q/q) + q, 'w') # as mentioned, can be manipulated symbolically

In [192]:
print w()

(1+k/2)/(1+k/2)+1+k/2


In [193]:
print 2 * q

2*(1+k/2)


For the most consistent results, any `QuantSpec` results that you want to use again should be defined into a proper `Var` or `Fun` object. Also, `QuantSpec`s are not a good place to manipulate vectors of symbols -- these should be converted to full Quantity types.

#Vectors as symbols

`Quantity` objects can define vectors of symbols (of up to rank 2). This can simplify notation, and permits `Fun` objects of several variables that can be symbolically differentiated. Examples of this are in `tests/Symbolic_Diff_test.py` and `tests/vode_withJac_Symbolic_test.py`.

In [201]:
x = dst.Var('x')
y = dst.Var('y')

f = dst.Var(['-3*Pow((2*x+1),3)+2*(x+y)', '-y/2'], 'f') # note that the definition is using a list!

print f[0] # only works if f is a Quantity type

-3*Pow((2*x+1),3)+2*(x+y)


In [208]:
f[0]

QuantSpec f_0 (ExpFuncSpec)

In [212]:
print f[1]

-y/2


Symbolic vectors can be defined for `QuantSpec`s and `Quantity`s, but are intended to be used in `Quantity` objects. This is because "array indexing" notation (using square brackets) is only properly supported for `Quantity`s. However, the `fromvector` method turns a vector `Quantity` or `QuantSpec` into a list of non-vector `QuantSpec` objects, each defining a component of the vector. It takes an optional integer index argument which selects that component of the vector only.

In [207]:
f.fromvector(0) # equivalent to f[1], but safe for both Quantities and QuantSpec



-3*Pow((2*x+1),3)+2*(x+y)


In [209]:
print f.fromvector(0)   

-3*Pow((2*x+1),3)+2*(x+y)


In [210]:
(f.fromvector(0))()

'-3*Pow((2*x+1),3)+2*(x+y)'

In [213]:
print f.fromvector()

[QuantSpec f_0 (ExpFuncSpec), QuantSpec f_1 (ExpFuncSpec)]


The method `tonumeric` reduces `Quantity`s whose definitions are entirely numeric to floating point numbers or arrays of such. 

In [222]:
df = dst.Diff(f, [x,y]) # differentiate each component of f with respect to x and y -- Jacobian, see next section
print df

[[-3*6*Pow((2*x+1),2)+2,2],[0,-0.5]]


In [217]:
dfe = df.eval(x=3,y=10)
print dfe

[[-880.0,2],[0,-0.5]]


In [218]:
dfe

QuantSpec __result__ (ExpFuncSpec)

In [219]:
dfe.tonumeric()

array([[ -8.80000000e+02,   2.00000000e+00],
       [  0.00000000e+00,  -5.00000000e-01]])

In [221]:
type(dfe.tonumeric())

numpy.ndarray

#Symbolic differentiation

Symbolic differentiation of mathematical expressions involving common mathematical functions is supported through the function `Diff`. 

The `Diff` function is meant to be a symbolic counterpart to the numerical derivative function `diff`, implemented in `common.py`, in the spirit of the Maple symbolic engine.

For many examples see the script `PyDSTool/tests/Symbolic_Diff_test.py`.

#Specification types

There are three specification sub-types of both Quantity and QuantSpec objects, determined by the 'specType' argument at initialization. These types are: 

* `'RHSfuncSpec'` (the named `Quantity` is defined as a right-hand side for differential or difference equation)
* `'ExpFuncSpec'` (the named `Quantity` is defined explicitly by the expression, such as for an auxiliary variable)
* `'ImpFuncSpec'` (the named `Quantity` is defined implicitly by the expression) 

The default is `'ExpFuncSpec'`. This type does not allow the `Quantity`'s name to appear in the defining expression, whereas the other two types do allow this.

In [234]:
f = dst.Var('2*k', 'f')

In [238]:
g = dst.Var('g(2*k)', 'g')

ValueError: Cannot define the symbol g in terms of itself with spec type ExpFuncSpec

In [241]:
g = dst.Var('g*2*k', 'g', specType="ImpFuncSpec")

In [244]:
h = dst.Var('h*2*k', 'h', specType="RHSfuncSpec")

These type strings are used by the `ModelConstructor` class to automatically determine how to treat each definition in a 'flattened' `ModelSpec` specification dictionary, and allows it to verify whether the types are compatible with the target `Generator` in which the specification will be instantiated. For instance, `'ImpFuncSpec'` definition types cannot be supplied to an `ExplicitFnGen` generator class

#Creation of Python functions from symbolic expressions

Symbolic expressions can be turned into Python functions using the `expr2fun` utility. The "free names" of an expression can either be numerically substituted from a supplied name -> value dictionary or left to become formal arguments to the function.

This utility can be handy to graph auxiliary functions from their abstract specifications (e.g., see `/examples/CIN.py`). Here is a simple example:

In [248]:
pyf = dst.expr2fun(f)
pyf

In [249]:
pyf(2)

4

In [250]:
vecpyf = np.vectorize(pyf)

In [251]:
vecpyf(np.arange(5))

array([0, 2, 4, 6, 8])